In [1]:
import torch
from torch.utils.cpp_extension import load

if torch.cuda.is_available():
    device = torch.device("cuda")

In [2]:
ultmul = load(name='ultmul', sources=['ultMul_cuda.cpp', 'ultMul_cuda_kernel.cu'])

import ultmul as um
um.ultmul

<function ultmul.PyCapsule.ultmul>

In [3]:
import sklearn.metrics.pairwise as sk_dist
torch.manual_seed(11000)
test = torch.randn(10001,10).to(device).float() # number of samples have to smaller than sqrt(512) need to fix
euc_dist = torch.from_numpy(sk_dist.euclidean_distances(test.to('cpu').numpy())).to(device).float()
print(euc_dist.size())
euc_dist

torch.Size([10001, 10001])


tensor([[0.0000, 3.8651, 3.7960,  ..., 5.0696, 4.0650, 3.8771],
        [3.8651, 0.0000, 3.6884,  ..., 3.6742, 3.8784, 3.2865],
        [3.7960, 3.6884, 0.0000,  ..., 5.0290, 3.1813, 2.8613],
        ...,
        [5.0696, 3.6742, 5.0290,  ..., 0.0000, 4.4713, 5.3355],
        [4.0650, 3.8784, 3.1813,  ..., 4.4713, 0.0000, 3.6948],
        [3.8771, 3.2865, 2.8613,  ..., 5.3355, 3.6948, 0.0000]],
       device='cuda:0')

In [4]:
#euc_dist
import time
torch.cuda.synchronize()   
start = time.clock()
u_test = um.ultmul(euc_dist, euc_dist)
torch.cuda.synchronize() 
end = time.clock()
tu = end-start
print(tu)

1.166117999999999


In [5]:
start = time.clock()
um.clusterability(euc_dist)
end = time.clock()
tc = end-start
print(tc)

50.750151


In [6]:
torch.cuda.synchronize()  
start = time.clock()
m_test = torch.matmul(euc_dist, euc_dist)
torch.cuda.synchronize()
end = time.clock()
tm = end-start
print(tm)

0.4080289999999991


In [7]:
usort = um.single_hclust(euc_dist, 3)
print(usort)

tensor([[   0, 6671, 6551,  ..., 7495, 5526, 6603],
        [6671,    0, 6671,  ..., 7495, 6671, 6671],
        [6551, 6671,    0,  ..., 7495, 6551, 6603],
        ...,
        [7495, 7495, 7495,  ...,    0, 7495, 7495],
        [5526, 6671, 6551,  ..., 7495,    0, 6603],
        [6603, 6671, 6603,  ..., 7495, 6603,    0]], device='cuda:0')


In [8]:
print(torch.max(euc_dist))
usort.size()

tensor(10.6978, device='cuda:0')


torch.Size([9995])

In [17]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
X = euc_dist.to('cpu').numpy()
print(X)

[[0.        3.8650737 3.796011  ... 5.069626  4.0649548 3.8770833]
 [3.8650737 0.        3.688441  ... 3.6742144 3.8784318 3.286531 ]
 [3.796011  3.688441  0.        ... 5.0290346 3.181317  2.8613245]
 ...
 [5.069626  3.6742144 5.0290346 ... 0.        4.4713006 5.335459 ]
 [4.0649548 3.8784318 3.181317  ... 4.4713006 0.        3.6948118]
 [3.8770833 3.286531  2.8613245 ... 5.335459  3.6948118 0.       ]]


In [18]:
clustering = AgglomerativeClustering(affinity = 'precomputed', linkage='single').fit(X)
print(clustering.labels_)

[0 0 0 ... 0 0 0]


In [19]:
print(clustering)

AgglomerativeClustering(affinity='precomputed', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='single', memory=None, n_clusters=2)
